<img style="width:40%;float: right;border-left:10px solid white" src="img/Monkeys.png"/>

### Baboon Crossing [14 points]

There is a deep canyon somewhere in Kruger National Park, South Africa, and a single rope that spans the canyon. Baboons can cross the canyon by swinging hand-over-hand on the rope from east to west or from west to east, but if two baboons going in opposite directions meet in the middle, they will fight and drop to their deaths. Furthermore, the rope is only strong enough to hold 5 baboons. If there are more baboons on the rope at the same time, it will break. Assuming that we can teach the baboons to use concurrency constructs, we would like to design a synchronization scheme with the following properties:
- Once a baboon has begun to cross, it is guaranteed to get to the other side without running into a baboon going the other way.
- There are never more than 5 baboons on the rope.
- A continuing stream of baboons crossing in one direction should not bar baboons going the other way indefinitely (no starvation).

[Adopted from Little Book of Semaphores by Allen B. Downey, 2016]

The rope has similarities to a critical section. Any solution to the critical section problem has to satisfy three safety properties and one liveness property:

1. Mutual exclusion
2. No deadlock (no livelock)
3. No unnecessary delay
4. Eventual entry

What are these four properties in terms of the Baboon Crossing problem? [2 points]

YOUR ANSWER HERE

Implement the Baboon Crossing problem in Java with the rope as a monitor by completing the template below. Your solution must satisfy all the four properties above to get full points. You must not modify the existing code, but you can add further code and you can add further cells for testing as you like. Only code in the cell below will be graded.

You only need to use `synchronized`, `wait`, `notify`, `notifyAll`, standard control structures and standard data types. You may ignore exceptions, for example by using `try {wait();} catch (Exception e) {}`. You must not import libraries (there is no need). [5 points]

_Hint:_ Answer this together with the questions below. You may introduce fields in `Rope` for
- the number of baboons crossing from East to West
- the number of baboons crossing from West to East
- the number of baboons waiting to cross from East to West
- the number of baboons waiting to cross from West to East
- the direction of the last baboons crossing



In [45]:
%%writefile baboons.java
class Rope {
    int num_EW;
    int num_WE;
    String prev_dir;

    Rope(){
        this.num_EW = 0; this.num_WE = 0; this.prev_dir = null;
    }

    synchronized void startEastToWest(int n){
        if(num_WE > 0 || num_EW >= 5 || "west".equals(prev_dir)) try{wait();} catch (Exception e) {}
        num_EW++;
        System.out.println("< going west" + n);
    }

    synchronized void finishEastToWest(int n){
        num_EW--;
        System.out.println(n + "< finished west");
        if(num_EW == 0) prev_dir = "west"; notifyAll(); 
    }

    synchronized void startWestToEast(int n){
        if(num_EW > 0 || num_WE >= 5 || "east".equals(prev_dir)) try{wait();} catch (Exception e) {}
        num_WE++;
        System.out.println(n + "going east >");  
    }

    synchronized void finishWestToEast(int n){
        num_WE--;
        System.out.println("finished east >" + n);
        if(num_WE == 0) prev_dir = "east"; notifyAll(); 
    }
}

class Baboon extends Thread {
    Rope r;
    int n;
    Baboon(Rope r, int n) {
        this.r = r; this.n = n; setDaemon(true);
    }
    public void run() {
        for (int i = 0; i < 1; i++) {
            r.startEastToWest(n);
            try {Thread.sleep(50);} catch (Exception e) {} // crossing to West side
            r.finishEastToWest(n);
            try {Thread.sleep(50);} catch (Exception e) {} // staying on West side
            r.startWestToEast(n);
            try {Thread.sleep(400);} catch (Exception e) {} // crossing to East side
            r.finishWestToEast(n);
            try {Thread.sleep(400);} catch (Exception e) {} // staying at East side
        }
    }
}

class Canyon {
    public static void main(String[] args) {
        Rope r = new Rope();
        for (int n = 0; n < 10; n++) {
            new Baboon(r, n).start();
        }
        try {Thread.sleep(5000);} catch (Exception e) {} // run program for 5 seconds
    }
}


Overwriting baboons.java


In [46]:
!javac baboons.java

In [47]:
!java Canyon

< going west0
< going west9
< going west8
< going west7
< going west6
0< finished west
< going west5
8< finished west
< going west1
< going west2
< going west3
6< finished west
< going west4
9< finished west
7< finished west
5< finished west
0going east >
1< finished west
8going east >
2< finished west
3< finished west
4< finished west
6going east >
7going east >
9going east >
finished east >0
5going east >
finished east >8
4going east >
3going east >
2going east >
1going east >
finished east >6
finished east >7
finished east >9
finished east >5
finished east >4
finished east >3
finished east >2
finished east >1


[Ungraded] For illustration, you may add statements

- `System.out.println("<" + n);` in `startEastToWest(int n)` when baboon `n` has started crossing,
- `System.out.println(n + "<");` in `finishEastToWest(int n)` when baboon `n` has finished crossing,
- `System.out.println(n + ">");` in `startWestToEast(int n)` when baboon `n` has started crossing,
- `System.out.println(">" + n);` in `finishWestToEast(int n)` when baboon `n` has finished crossing.

In that case, a possible fragment of the output is:
```
<5
<9
<8
<7
<6
5<
9<
7<
8<
6<
0>
>0
```

Consider following monitor in algorithmic notation. What are the conditions for the monitor to be correct, i.e. `I` to be an invariant? [2 points]

```algorithm
monitor M
    var a
    {I}
    initialization(v₀)
        {P₀}  S₀  {Q₀}
    procedures pᵢ(vᵢ) → (rᵢ)
        {Pᵢ}  Sᵢ  {Qᵢ}
```

YOUR ANSWER HERE

Suppose monitor `M` declares a local condition variable `cv` and the monitor procedures contain statements `wait(cv)` and `signal(cv)`. What are the correctness rules `wait(cv)` and `signal(cv)` with the signal-and-continue discipline? Give the rules and a very brief explanation. [2 points]

YOUR ANSWER HERE

Give the strongest invariant of the monitor `Rope`. You may use mathematical notation or Java notation, as long as you are consistent (and it is clear what equality is). [2 points]

**Symbols to copy and paste:** ⟨ ⟩ ¬ ∧ ∨ ≠ ≡ ⇒ ⇐ ≤ ≥

YOUR ANSWER HERE

Can baboons crossing in the same direction overtake each other? Answer Yes or No and give a brief explanation. [1 point]

YOUR ANSWER HERE